In [1]:
import torch
import torch.nn as nn
from src.config import get_lora_config
from src.config import add_task_to_seq_lora_config, get_seq_lora_config
from src.models import SequentialLoRAModel

from transformers import AutoModelForSequenceClassification, AutoTokenizer

c:\Users\nikhi\Documents\Projects\LoRA-Ensembling\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
config = get_seq_lora_config("blabla", "qv")
config = add_task_to_seq_lora_config(config, "sst2")

model = SequentialLoRAModel(config)

2024-07-03 03:27:05.358 | DEBUG    | src.models.sequential_lora_for_classification:__init__:128 - Loading model bert-base-uncased...
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-07-03 03:27:07.193 | DEBUG    | src.models.sequential_lora_for_classification:__init__:139 - Model loaded successfully.
2024-07-03 03:27:07.193 | DEBUG    | src.models.sequential_lora_for_classification:__init__:140 - Injecting LoRA layers...
2024-07-03 03:27:07.195 | DEBUG    | src.models.sequential_lora_for_classification:__init__:143 - Add task to LoRA
2024-07-03 03:27:07.195 | DEBUG    | src.models.sequential_lora_for_classification:add_task:259 - Adding the task: sst2
2024-07-03 03:27:07.199 | DEBUG    | src.models.sequential_lora_for_classification:set_curr

In [6]:
for param in model.model.bert.encoder.layer[0].attention.self.query.lora.lora_As:
    print(param.requires_grad)

True


In [25]:
model.config = add_task_to_seq_lora_config(model.config, "cola")
model.add_task("cola")

2024-07-03 18:54:36.811 | DEBUG    | src.models.sequential_lora_for_classification:add_task:259 - Adding the task: cola


In [3]:
from src.data import get_datamodule

sst2 = get_datamodule("sst2")("bert-base-uncased", 8, 4)

sst2.prepare_data()
sst2.setup()

In [4]:
from pytorch_lightning import Trainer

trainer = Trainer(
    max_epochs=config.training.max_epochs,
    accelerator=config.training.accelerator,
    precision=config.training.precision,
    deterministic=True,
    val_check_interval=config.training.val_check_interval,
)

trainer.fit(model, datamodule=sst2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\nikhi\Documents\Projects\LoRA-Ensembling\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/

Epoch 0:  76%|███████▋  | 6423/8418 [5:27:14<1:41:38,  0.33it/s, v_num=20, train/loss=0.700, train/acc=0.500, val/loss=0.698, val/acc=0.509] 

c:\Users\nikhi\Documents\Projects\LoRA-Ensembling\.venv\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [25]:
trainloader = sst2.train_dataloader()

batch = next(iter(trainloader))
batch

{'labels': tensor([1, 0, 1, 1, 0, 1, 1, 1]), 'input_ids': tensor([[  101,  2003, 19957,  2864,  2011,  1037, 18109,  1997, 19910,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  2130,  2295,  1996,  2143,  2515,  1050,  1005,  1056,  6133,
          2000,  2718,  2035,  1997,  2049,  6017,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  7467,  1996, 17170,  6562,  2166,  2006,  1996, 21484,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101, 16481,  2016,  2038,  1996,  4933,  2000,  3233,  4206,  2007,
         29198,  2953,  1010,  5529,  